# 1.0 Example - Correlating Process Paramters to XAS Spectra Time Steps

Time-dependent process parameters (Mass Spec, LabView) will be interpolated onto the time-grid of the XAS data making for quick spectra-process parameters plotting and lookup

Correlated data is retunred as a pandas dataframe and saved as a .csv file.

### Things to have ready when attemtping this example:
1. Directoty containing only XAS spectra
2. Data structure of the XAS files, key paramters are where the time stamp is stired
3. Path and file name for process paramters files (MS and LV)
4. Directory where to write the summary file

## Step 1. Import Modules

In [1]:
# File Handling
import os

# Math and Data Handling
import pandas as pd

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp

C:\Users\ashoff\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


ModuleNotFoundError: No module named 'larch'

## Step 2. Path Definitions

Specify paths to XAS data, process data streams, and output files

#### Notes:
1. data_dir can only contain XAS spectra that were collected in the same manner (e.g. steady state scans, or all in-situ scan)
2. *_filename paths must be the entire path with file extension
3. output_dir must be an existing folder
4. result_fname should not have an extension


In [ ]:
# Folder containing ONLY CXAS spectra
data_dir = os.path.join(pwd, 'sample data\Raw Data') 

# Mass Spec File:
MS_filename = os.path.join(pwd, 'sample data\SnO2_TPR_MS.csv') 

# AmP LabView File:
LV_filename = os.path.join(pwd, 'sample data\SnO2_TPR_LV.txt')

# Folder where the correlated datatable will be written
output_dir = os.path.join(pwd, 'sample results')

# Name of fole to be written
result_fname = 'SnO2_TPR_Process_Correlation'

## Step 3. Define Beamline Data Structure

Common data formats for SSRL beamlines can be found in the "BL specific XAS data structures.ipynb" notebook.

For this example the key values needed are 'time stamp' (must be true), 'time on line', 'time format', 'padded scan numbers' (padded = leading zeros before the scan number, e.g. *_001 instead of *_1), and 'is QEXAFS'

In [ ]:
### Define the layout of the xas data file and how spectra are calcualted
xas_data_structure = {
    'time stamp': True,
    'time on line': 5,
    'time format': '# This Scan Create Date:\t%m/%d/%Y %I:%M:%S %p ',
    'padded scan numbers': True,
    'column names': ['Encoder', 'Energy', 'ADC_01', 'ADC_02', 'ADC_03', 'ADC_04', 'ADC_05', 'ADC_06', 'ADC_07', 'ADC_08'],
    'energy column' : 'Energy', # Energy [eV]
    'sample numerator': 'ADC_01', # I0
    'sample denominator': 'ADC_02', # I1
    'sample ln': True,
    'sample invert': False,
    'reference numerator': 'ADC_02', # I1
    'reference denominator': 'ADC_03', # I2
    'reference ln': True,
    'reference invert': False,
    'is QEXAFS': False
}

## Step 4. Summarize file names and time steps of XAS spectra 

### DO NOT MODIFY THIS SECTION

In [ ]:
time_stamp = xas_data_structure['time stamp']
time_line = xas_data_structure['time on line']
time_format = xas_data_structure['time format']
padded = xas_data_structure['padded scan numbers']
is_QEXAFS = xas_data_structure['is QEXAFS']


XAS_summary = xfcts.CXAS_Sorted(data_dir,
                   time_stamp=time_stamp, time_line=time_line, time_format=time_format,
                   padded=padded, is_QEXAFS = is_QEXAFS)

In [ ]:
# Visualize the summary fole of the XAS data 
XAS_summary

## Step 5. Import Process Parameter Files 

The catxas.general_functions module has a list of fuctions for reading different process files. More formats can be added as needed

### MODIFY by:
Adding additional lines to read in all process variable streams

In [ ]:
MS_data = process.ReadMSData(MS_filename)

LV_data = process.ReadLVData(LV_filename)

## Step 6. Interpolate Process Paramters onto XAS spectra Timestamps

### MODIFY by:
1. Adding or removing functions to account for all process variable streams
2. Update list of dataframes that will be joined in the output file

In [ ]:
MS_interp = fcts.mergeindex(XAS_summary, MS_data) 
LV_interp = fcts.mergeindex(XAS_summary, LV_data)

concat_list = [XAS_summary['TOS [s]'],XAS_summary['File Name'], MS_interp, LV_interp]

## Step 7. Concatenate XAS Filename and Time Summar with Process Parameters

### DO NOT MODIFY THIS SECTION

In [ ]:
process_params = pd.concat(concat_list, axis = 1)

In [ ]:
# Visualize the xas-process parameters summary dataframe 
process_params

## Step 8. Save Dataframe to CSV file

### DO NOT MODIFY THIS SECTION

In [ ]:
process_params.to_csv(os.path.join(output_dir, result_fname + '.csv'), sep=',', 
                      na_rep='', header=True, index=True, index_label='Time')